# COVID-19 DETECTOR USING IMAGE OF CHEST-XRAY.

In [32]:
import os #for access directory and folder of operating system
import numpy as np
import csv
import pandas
import cv2 #for image conversion means convert into pixle
import matplotlib.pyplot as plt #for show image that are converted into the array of pixel
training_data=[] #Traing data will store into that list
path = 'C:/Users/om/Desktop/covid-chestxray-dataset-master/images'
result = pandas.read_csv('C:/Users/om/Desktop/covid-chestxray-dataset-master/metadata.csv')
for l in range(0,len(result)):
    img_array = cv2.imread(os.path.join(path,result['filename'][l]),cv2.IMREAD_GRAYSCALE) #convert jpg to pixle of array and also convert into grayscale...........
    new_array = cv2.resize(img_array,(50,50)) #50,50 -> convert size of image into 50*50 pixel
    class_name = result['finding'][l] #label's
    if(class_name=="COVID-19"):
        class_name=1 
    else:
        class_name=0
    training_data.append([new_array,class_name])
print(len(training_data))

171


In [33]:
import random #for shuffle all the traiing_input array
random.shuffle(training_data)

In [34]:
X = []
Y = []
for features,label in training_data:
    X.append(features)
    Y.append(label)
X = np.array(X).reshape(-1,50,50,1) # 1-> here image is gray scale so write 1 , if ur image is rgb then write  3.
Y = np.array(Y)

X = X/255.0 #Reduce pixle size
print(Y[151])

0


In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(50, 50, 1))) # 32->number of nuerons, (3,3)->window size of input, (50,50,1)->image size 50*50 and 1 because image is gray scale....
model.add(Activation('relu')) #Activation function
model.add(MaxPooling2D(pool_size=(2, 2))) #MaxPooling2D is layer which select maximum size window and combine into new 2*2 array so pool_size is (2,2).......

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())# this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64)) # (64)->nuerons in Dense layer
model.add(Activation('relu'))

# model.add(Dropout(0.5))

model.add(Dense(2)) # final 1 output layer
model.add(Activation('softmax'))
# COMPILEself
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])#sparse_categorycal_crossentropy=if more than one class,categorycal_crossentropy=if predict only one class



In [37]:
model.fit(X,Y,epochs=18,batch_size=5,validation_split=0.1) #batch_size=total number of input pass at a time, validation_split=(0.1)10% of input are use for validation......

Train on 153 samples, validate on 18 samples
Epoch 1/18
153/153 [==============================] - 1s 4ms/sample - loss: 0.2246 - accuracy: 0.9477 - val_loss: 0.8060 - val_accuracy: 0.8333
Epoch 2/18
153/153 [==============================] - 1s 5ms/sample - loss: 0.1683 - accuracy: 0.9346 - val_loss: 0.8820 - val_accuracy: 0.5556
Epoch 3/18
153/153 [==============================] - 1s 5ms/sample - loss: 0.1828 - accuracy: 0.9020 - val_loss: 0.7183 - val_accuracy: 0.7778
Epoch 4/18
153/153 [==============================] - 1s 5ms/sample - loss: 0.1543 - accuracy: 0.9346 - val_loss: 1.1721 - val_accuracy: 0.7778
Epoch 5/18
153/153 [==============================] - 1s 5ms/sample - loss: 0.4002 - accuracy: 0.8431 - val_loss: 0.6036 - val_accuracy: 0.7778
Epoch 6/18
153/153 [==============================] - 1s 5ms/sample - loss: 0.2194 - accuracy: 0.9020 - val_loss: 0.7127 - val_accuracy: 0.8333
Epoch 7/18
153/153 [==============================] - 1s 5ms/sample - loss: 0.1289 - accura

In [41]:
import cv2
import numpy as np
import tensorflow as tf
def convert_image(img):
    ia = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
    na = cv2.resize(ia,(50,50))
    return na.reshape(-1,50,50,1)

t  = np.array(convert_image('C:/Users/om/Desktop/1.jpg'))
t = tf.cast(t, tf.float32)
p = model.predict(t)
if(p[0][0]==1.0):
    print("Not infected by coronavirus")
else:
    print("Infected by coronavirus")

Infected by coronavirus


In [9]:
!pip install h5py


In [42]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [1]:
from tensorflow.python.keras.models import model_from_json
import keras
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# score = loaded_model.evaluate(X, Y, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))


Using TensorFlow backend.


Loaded model from disk


In [5]:
import cv2
import numpy as np
import tensorflow as tf
def convert_image(img):
    ia = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
    na = cv2.resize(ia,(50,50))
    return na.reshape(-1,50,50,1)

t  = np.array(convert_image('C:/Users/om/Desktop/3.jpg'))
t = tf.cast(t, tf.float32)
p = loaded_model.predict(t)
print(p)
if(p[0][0]==1.0):
    print("Not infected by coronavirus")
else:
    print("Infected by coronavirus")

[[1. 0.]]
Not infected by coronavirus
